In [1]:
from tensorflow.keras.models import load_model

In [2]:
model = load_model('NeuralNetwork_sentiment_model.h5')

In [3]:
import requests
from datetime import datetime, timedelta
import numpy as np

In [4]:
API_KEY = 'ab7ba0cc07f94b00a1237576550365b5'
url = 'https://newsapi.org/v2/everything'

In [32]:
end_date = datetime.now()
start_date = end_date - timedelta(days=2)


In [33]:
params = {
    'q': 'Reliance',  # Your search query
    'from': start_date.strftime('%Y-%m-%d'),  # From date
    'to': end_date.strftime('%Y-%m-%d'),  # To date
    'sortBy': 'publishedAt',
    'apiKey': API_KEY,
}

In [34]:
response = requests.get(url, params=params)
articles = response.json().get('articles', [])

In [35]:
news_texts = []
for article in articles:
    news_texts.append(article['title']) # + ' ' + article['description'])

In [36]:
news_texts

['Telcos creating a false equivalence with satellite operators on spectrum pricing : Amazon, Starlink',
 "Puitn's Nightmare: Russian Soldiers and Civilians are Turning on the Ukraine War",
 "Pete Prisco's Week 9 NFL picks: Lions knock off Packers in NFC North showdown; surging Rams topple Seahawks",
 '"Election" "Day" Minus Two',
 'Energy, automotive, banking sectors offer most of internships under PM scheme',
 'Ethiopia bans imports of gas-powered private vehicles, but the switch to electric is a bumpy ride',
 'I, Coppola',
 'The Indispensable Imperative: Institutions Must Do More To Mitigate Cybersecurity Risks',
 '[Removed]',
 'Private Calls and Campaign Cash: Louisiana Regulators’ Ties to the Utilities They Oversee',
 'U.S. DOE Set to Award $150 MM for Energy Upgrades at Federal Facilities',
 'What Trump has said he would do on Day 1 back in the White House',
 'Beware AI Voting Assistants: Impact On Swing States In The 2024 U.S. Election',
 'Will politicians never learn? Subsidised

In [67]:
filtered_news = [entry for entry in news_texts if "[Removed]" not in entry]

In [68]:
filtered_news

['Telcos creating a false equivalence with satellite operators on spectrum pricing : Amazon, Starlink',
 "Puitn's Nightmare: Russian Soldiers and Civilians are Turning on the Ukraine War",
 "Pete Prisco's Week 9 NFL picks: Lions knock off Packers in NFC North showdown; surging Rams topple Seahawks",
 '"Election" "Day" Minus Two',
 'Energy, automotive, banking sectors offer most of internships under PM scheme',
 'Ethiopia bans imports of gas-powered private vehicles, but the switch to electric is a bumpy ride',
 'I, Coppola',
 'The Indispensable Imperative: Institutions Must Do More To Mitigate Cybersecurity Risks',
 'Private Calls and Campaign Cash: Louisiana Regulators’ Ties to the Utilities They Oversee',
 'U.S. DOE Set to Award $150 MM for Energy Upgrades at Federal Facilities',
 'What Trump has said he would do on Day 1 back in the White House',
 'Beware AI Voting Assistants: Impact On Swing States In The 2024 U.S. Election',
 'Will politicians never learn? Subsidised trains are fo

In [69]:
filtered_news = [title for title in filtered_news if 'reliance' in title.lower()]


In [70]:
filtered_news

['Reliance, Inc. (NYSE:RS) Shares Sold by Wealth Enhancement Advisory Services LLC']

In [71]:
final_news=filtered_news

In [41]:
def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = ''.join(char for char in text if char.isalpha() or char.isspace())
    return text

In [42]:
filtered_news = [normalize_text(news) for news in filtered_news]

In [43]:
filtered_news

['reliance inc nysers shares sold by wealth enhancement advisory services llc']

In [44]:
from nltk.corpus import stopwords

In [45]:
stop_words = set(stopwords.words('english'))

In [46]:
filtered_news = [[word for word in news.split() if word not in stop_words] for news in filtered_news]

In [47]:
from nltk.stem import PorterStemmer

In [48]:
stemmer = PorterStemmer()

In [49]:
filtered_news = [[stemmer.stem(word) for word in news] for news in filtered_news]


In [50]:
filtered_news = [' '.join(news) for news in filtered_news]

In [51]:
filtered_news

['relianc inc nyser share sold wealth enhanc advisori servic llc']

In [25]:
# from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# tfidf_vectorizer1 = TfidfVectorizer() 

In [27]:
# text_tfidf=tfidf_vectorizer1.fit_transform(filtered_news).toarray()

In [28]:
# text_tfidf

In [52]:
import joblib

In [53]:
tfidf_vectorizer = joblib.load('tfidf_vectorizer_NeuralNetwork1.pkl')

In [54]:
X_new_tfidf = tfidf_vectorizer.transform(filtered_news)

In [55]:
X_new_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8 stored elements and shape (1, 10419)>

In [56]:
X_new_tfidf.shape

(1, 10419)

In [57]:
predictions = model.predict(X_new_tfidf)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


In [58]:
label_encoder = joblib.load('label_encoder_NeuralNetwork.pkl')

In [59]:
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

In [ ]:
confidence_score = np.max(predictions, axis=1)  # max probability as confidence

In [73]:
for text, pred, score in zip(final_news, predicted_labels, confidence_score):
    print(f'News Title: "{text}" - Predicted Sentiment: {pred} (Confidence: {score:.2f})')

News Title: "Reliance, Inc. (NYSE:RS) Shares Sold by Wealth Enhancement Advisory Services LLC" - Predicted Sentiment: negative (Confidence: 0.99)
